Created on November 22, 2024

Course work: 

@author: Madhuri


In [1]:
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import datetime

In [2]:
end = datetime.date.today()

begin=end-pd.DateOffset(365*10)

st=begin.strftime('%Y-%m-%d')

ed=end.strftime('%Y-%m-%d')


df = pdr.get_data_google("AAPL",st,ed)


no_of_std = 12



def bollinger_strat(df,window,std):
    rolling_mean = df['Close'].rolling(window).mean()
    rolling_std = df['Close'].rolling(window).std()

    df['Bollinger High'] = rolling_mean + (rolling_std * no_of_std)
    df['Bollinger Low'] = rolling_mean - (rolling_std * no_of_std)


data = [1, 2, 4, 5, 5, 3]

bollinger_strat(data,20,2)

AttributeError: module 'pandas_datareader.data' has no attribute 'get_data_google'